# Import Library

### 1. Library dan Dependensi Utama

Proyek ini memanfaatkan berbagai teknologi modern yang saling terintegrasi untuk mendukung proses pemrosesan data, pemodelan, dan pencarian berbasis vektor.

- **Manipulasi Data**  
  `pandas` dan `numpy` digunakan untuk pengolahan data berbasis *dataframe* serta komputasi numerik.
- **Machine Learning (Scikit-learn)**  
  - `TfidfVectorizer` untuk mengekstraksi fitur dari data teks.  
  - `SelectKBest` untuk melakukan seleksi fitur yang paling relevan.  
  - `LabelEncoder` dan `MinMaxScaler` untuk prapemrosesan serta normalisasi data.
- **Deep Learning (TensorFlow/Keras)**  
  Digunakan untuk membangun dan melatih arsitektur *neural network* pada tugas klasifikasi.
- **Text Embedding (Sentence-Transformers)**  
  Mengonversi teks menjadi representasi vektor semantik menggunakan model berbasis BERT.
- **Vector Search (FAISS)**  
  Library dari Meta yang digunakan untuk pencarian kemiripan (*similarity search*) berbasis vektor secara efisien dalam skala besar.
- **Persistensi Model**  
  `pickle` dan `joblib` digunakan untuk menyimpan serta memuat kembali model dan artefak yang telah dilatih.

---

### 2. Alur Kerja (Workflow)

- **Prapemrosesan Data**  
  Membersihkan data mentah dan mengubah teks menjadi representasi numerik menggunakan TF-IDF atau *text embeddings*.
- **Feature Engineering**  
  Melakukan seleksi fitur yang paling informatif serta normalisasi skala data untuk meningkatkan performa model.
- **Pembangunan Model**  
  Melatih model klasifikasi menggunakan TensorFlow berdasarkan fitur yang telah diproses.
- **Indexing dan Retrieval**  
  Menyimpan vektor fitur ke dalam indeks FAISS untuk mendukung proses pencarian dan pengambilan data secara cepat dan efisien.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import faiss
import pickle
import joblib
import ast

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras import layers, models
from sentence_transformers import SentenceTransformer

# Prepare Data

### Pemuatan dan Inspeksi Dataset

Tahap awal ini bertujuan untuk memuat dataset hasil prapemrosesan ke dalam *DataFrame* serta melakukan pemeriksaan awal terhadap struktur dan isi data.

- **Sumber Data:**  
  `data17000.csv`, yaitu dataset yang telah melalui proses ekstraksi skill.

- **Tujuan:**  
  - Memastikan seluruh kolom yang dibutuhkan (seperti teks mentah, label, dan fitur numerik) telah dimuat dengan benar.  
  - Melakukan inspeksi awal terhadap data menggunakan `df.head()` untuk menampilkan lima baris pertama.

> **Catatan:**  
> Dataset ini akan diproses lebih lanjut menggunakan `LabelEncoder` untuk pengodean label kategorikal, serta `TfidfVectorizer` atau `SentenceTransformer` untuk ekstraksi fitur teks.

In [2]:
df = pd.read_csv("../data/data17000.csv")
df.head()

,Unnamed: 0,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary,job_description_cleaned,skills
0,0,1,Facility Maintenance & Smart Warehouse Manager,Bandung,IDR,Manajer/Asisten Manajer,5 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Manufaktur,Pemeliharaan",NaN,NaN,NaN,NaN,Deskripsi PekerjaanRequirements :D3/SI from re...,NaN,deskripsi pekerjaanrequirements si from reputa...,"['electrical inspection', 'management system',..."
1,1,2,Procurement Department Head,Jakarta Raya,IDR,Manajer/Asisten Manajer,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Manufaktur,Pembelian/Manajemen Material",NaN,25 days,51 - 200 pekerja,Manajemen/Konsulting HR,Job Role: 1. Responsible for material availabi...,NaN,job role responsible for material availabili...,"['heavy equipment', 'contract management', 'pr..."
2,2,3,SALES ADMIN,Jakarta Barat,IDR,Supervisor/Koordinator,4 tahun,Sarjana (S1),Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",30 days,51 - 200 pekerja,Umum & Grosir,Internal Sales & AdminJob Description :We are ...,NaN,internal sales adminjob description we are loo...,"['microsoft office', 'heat exchanger', 'carbon..."
3,3,4,City Operation Lead Shopee Express (Cirebon),Cirebon,IDR,Supervisor/Koordinator,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Pelayanan,Logistik/Rantai Pasokan","Tip;Waktu regular, Senin - Jumat;Kasual (conto...",21 days,2001 - 5000 pekerja,Retail/Merchandise,Job Description:Responsible for HSE implementa...,NaN,job description responsible for hse implementa...,"['operation management', 'analytical skill', '..."
4,4,5,Japanese Interpreter,Bekasi,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Lainnya,Jurnalis/Editor",NaN,23 days,201 - 500 pekerja,Manajemen/Konsulting HR,Overview: Our clients is manufacture for autom...,NaN,overview our clients is manufacture for automo...,"['japanese', 'translator', 'english', 'non', '..."


## Inspeksi Metadata Dataset

Metode `df.info()` digunakan untuk memperoleh gambaran teknis DataFrame secara menyeluruh. Informasi ini membantu memahami struktur data sebelum dilakukan analisis atau pemodelan lebih lanjut. Detail yang ditampilkan meliputi:

- **Rentang Indeks**  
  Menunjukkan jumlah total baris atau entri yang terdapat dalam dataset.
- **Daftar Kolom**  
  Menampilkan seluruh nama kolom yang tersedia di dalam DataFrame.
- **Non-Null Count**  
  Menunjukkan jumlah nilai yang terisi pada setiap kolom, sehingga memudahkan identifikasi *missing values* (data kosong).
- **Dtype (Tipe Data)**  
  Menginformasikan tipe data pada masing-masing kolom, seperti `int64`, `float64`, atau `object` untuk data teks.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17182 entries, 0 to 17181
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               17182 non-null  int64  
 1   id                       17182 non-null  int64  
 2   job_title                17182 non-null  object 
 3   location                 17182 non-null  object 
 4   salary_currency          17178 non-null  object 
 5   career_level             17182 non-null  object 
 6   experience_level         17182 non-null  object 
 7   education_level          17182 non-null  object 
 8   employment_type          17182 non-null  object 
 9   job_function             17182 non-null  object 
 10  job_benefits             14440 non-null  object 
 11  company_process_time     13377 non-null  object 
 12  company_size             15488 non-null  object 
 13  company_industry         16412 non-null  object 
 14  job_description       

## Identifikasi Nilai Unik pada Kolom

Bagian ini digunakan untuk mengidentifikasi kategori yang terdapat pada suatu fitur serta mengetahui jumlah kelas yang ada di dalamnya.

- **`df['job_function'].unique()`**  
  Mengambil seluruh nilai unik (tanpa duplikasi) dari kolom `job_function` dan menyimpannya dalam bentuk array NumPy.
- **`len(unique_values_list)`**  
  Menghitung total jumlah kategori unik yang teridentifikasi. Informasi ini penting dalam tugas klasifikasi karena menentukan jumlah kelas atau label yang akan diprediksi oleh model.

In [4]:
# Get a NumPy array of all unique values
unique_values_list = df['job_function'].unique()
print(len(unique_values_list))

68


## Mapping dan Pengelompokan Kategori

Bagian ini bertujuan menyederhanakan target prediksi dengan mengelompokkan subkategori yang bersifat spesifik ke dalam kategori yang lebih umum dan representatif.

- **`job_function_mapping`**  
  Sebuah *dictionary* yang berfungsi sebagai kamus pemetaan kategori. Sebagai contoh, kategori *Audit & Pajak* dan *Perbankan* digabungkan ke dalam satu kelompok, yaitu **Finance & Accounting**.
- **`.map()`**  
  Metode yang digunakan untuk memetakan setiap nilai pada kolom `job_function` berdasarkan pasangan kunci–nilai yang didefinisikan di dalam *dictionary* pemetaan.
- **`.fillna("Other")`**  
  Digunakan untuk menangani nilai yang tidak memiliki pasangan dalam kamus pemetaan atau data yang kosong, dengan menetapkan label bawaan **Other**.
- **`df["job_function_group"]`**  
  Kolom baru hasil transformasi pemetaan kategori. Pendekatan ini membantu model mempelajari pola data dengan lebih baik karena jumlah kelas (*label*) menjadi lebih sedikit, terstruktur, dan seimbang.

In [5]:
job_function_mapping = {

    # Finance & Accounting
    "Akuntansi / Keuangan,Audit & Pajak": "Finance & Accounting",
    "Akuntansi / Keuangan,Akuntansi Umum / Pembiayaan": "Finance & Accounting",
    "Akuntansi / Keuangan,Perbankan / Jasa Finansial ": "Finance & Accounting",
    "Akuntansi / Keuangan,Keuangan / Investasi Perusahaan ": "Finance & Accounting",
    "Sains,Aktuaria/Statistik": "Finance & Accounting",

    # IT & Software
    "Komputer/Teknologi Informasi,IT-Perangkat Lunak": "IT & Software",
    "Komputer/Teknologi Informasi,IT-Admin Jaringan/Sistem/Database": "IT & Software",
    "Komputer/Teknologi Informasi,IT-Perangkat Keras": "IT & Software",

    # Engineering (Non-IT)
    "Teknik,Teknik Lingkungan": "Engineering",
    "Teknik,Teknik Lainnya": "Engineering",
    "Teknik,Mekanikal": "Engineering",
    "Teknik,Teknik Elektronika": "Engineering",
    "Teknik,Teknik Elektro": "Engineering",
    "Teknik,Teknik Industri": "Engineering",
    "Teknik,Teknik Kimia": "Engineering",
    "Bangunan/Konstruksi,Teknik Sipil/Konstruksi Bangunan": "Engineering",
    "Bangunan/Konstruksi,Survei Kuantitas": "Engineering",
    "Sains,Pertanian": "Engineering",
    "Sains,Teknologi Makanan/Ahli Gizi": "Engineering",
    "Sains,Sains & Teknologi": "Engineering",

    # Manufacturing & Operations
    "Manufaktur,Manufaktur": "Manufacturing & Operations",
    "Manufaktur,Pemeliharaan": "Manufacturing & Operations",
    "Manufaktur,Pembelian/Manajemen Material": "Manufacturing & Operations",
    "Manufaktur,Penjaminan Kualitas / QA": "Manufacturing & Operations",
    "Pelayanan,Logistik/Rantai Pasokan": "Manufacturing & Operations",

    # Sales & Marketing
    "Penjualan / Pemasaran,Pemasaran/Pengembangan Bisnis": "Sales & Marketing",
    "Penjualan / Pemasaran,Digital Marketing": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Jasa Keuangan": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Teknik/Teknikal/IT": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan Ritel": "Sales & Marketing",
    "Penjualan / Pemasaran,Merchandising": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Korporasi": "Sales & Marketing",
    "Penjualan / Pemasaran,Telesales/Telemarketing": "Sales & Marketing",
    "Penjualan / Pemasaran,E-commerce": "Sales & Marketing",
    "Seni/Media/Komunikasi,Periklanan": "Sales & Marketing",

    # HR & Administration
    "Sumber Daya Manusia/Personalia,Sumber Daya Manusia / HR": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Staf / Administrasi umum": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Sekretaris": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Top Management / Manajemen Tingkat Atas": "HR & Administration",
    "Pelayanan,Pengacara / Asisten Legal": "HR & Administration",
    "Pelayanan,Angkatan Bersenjata": "HR & Administration",

    # Creative & Media
    "Seni/Media/Komunikasi,Seni / Desain Kreatif": "Creative & Media",
    "Seni/Media/Komunikasi,Hubungan Masyarakat": "Creative & Media",
    "Seni/Media/Komunikasi,Hiburan": "Creative & Media",
    "Lainnya,Jurnalis/Editor": "Creative & Media",

    # Education & Training
    "Pendidikan/Pelatihan,Pendidikan": "Education",
    "Pendidikan/Pelatihan,Pelatihan & Pengembangan": "Education",

    # Hospitality & Service
    "Hotel/Restoran,Makanan/Minuman/Pelayanan Restoran": "Hospitality & Service",
    "Hotel/Restoran,Hotel/Pariwisata": "Hospitality & Service",
    "Pelayanan,Layanan Pelanggan": "Hospitality & Service",
    "Pelayanan,Teknikal & Bantuan Pelanggan": "Hospitality & Service",
    "Pelayanan,Perawatan Pribadi": "Hospitality & Service",
    "Layanan Kesehatan,Diagnosa/Lainnya": "Hospitality & Service",
    "Layanan Kesehatan,Farmasi": "Hospitality & Service",
    "Layanan Kesehatan,Praktisi/Asisten Medis": "Hospitality & Service",
    "Sains,Bioteknologi": "Hospitality & Service",
    "Sains,Biomedis": "Hospitality & Service",
    "Sains,Kimia": "Hospitality & Service",
    "Sains,Geologi/Geofisika": "Hospitality & Service",

    # Other
    "Bangunan/Konstruksi,Arsitek/Desain Interior": "Other",
    "Bangunan/Konstruksi,Properti/Real Estate": "Other",
    "Lainnya,Pekerjaan Umum": "Other",
    "Lainnya,Lainnya/Kategori tidak tersedia": "Other",
}

df["job_function_group"] = (
    df["job_function"]
    .map(job_function_mapping)
    .fillna("Other")
)

## Label Encoding untuk Target Kategorikal

Tahap ini bertujuan mengonversi data kategorikal berbasis teks ke dalam representasi numerik agar dapat diproses oleh algoritma *machine learning* maupun *deep learning*.

- **`LabelEncoder()`**  
  Merupakan utilitas dari Scikit-learn yang memetakan setiap kategori unik ke dalam nilai numerik tertentu, misalnya `"Engineering"` menjadi `0`, `"Finance"` menjadi `1`, dan seterusnya.
- **`fit_transform()`**  
  Menjalankan dua proses sekaligus, yaitu mempelajari seluruh kategori unik yang terdapat pada kolom target (*fit*) dan mengonversinya langsung ke bentuk numerik (*transform*).
- **`encoders_dict`**  
  Sebuah *dictionary* yang digunakan untuk menyimpan objek encoder. Penyimpanan ini penting agar proses *inverse transform*—mengembalikan prediksi numerik ke label teks—dapat dilakukan dengan benar.
- **Penyimpanan Metadata**  
  Dengan menyimpan encoder di dalam `encoders_dict`, konsistensi pemetaan label antara data pelatihan dan data baru tetap terjaga, terutama pada tahap inferensi dan *deployment* model.

In [6]:
encoders_dict = {}

columns_to_encode = ['job_function_group']
for col in columns_to_encode:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders_dict[col] = le

df.head()

,Unnamed: 0,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary,job_description_cleaned,skills,job_function_group
0,0,1,Facility Maintenance & Smart Warehouse Manager,Bandung,IDR,Manajer/Asisten Manajer,5 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Manufaktur,Pemeliharaan",NaN,NaN,NaN,NaN,Deskripsi PekerjaanRequirements :D3/SI from re...,NaN,deskripsi pekerjaanrequirements si from reputa...,"['electrical inspection', 'management system',...",7
1,1,2,Procurement Department Head,Jakarta Raya,IDR,Manajer/Asisten Manajer,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Manufaktur,Pembelian/Manajemen Material",NaN,25 days,51 - 200 pekerja,Manajemen/Konsulting HR,Job Role: 1. Responsible for material availabi...,NaN,job role responsible for material availabili...,"['heavy equipment', 'contract management', 'pr...",7
2,2,3,SALES ADMIN,Jakarta Barat,IDR,Supervisor/Koordinator,4 tahun,Sarjana (S1),Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",30 days,51 - 200 pekerja,Umum & Grosir,Internal Sales & AdminJob Description :We are ...,NaN,internal sales adminjob description we are loo...,"['microsoft office', 'heat exchanger', 'carbon...",9
3,3,4,City Operation Lead Shopee Express (Cirebon),Cirebon,IDR,Supervisor/Koordinator,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Pelayanan,Logistik/Rantai Pasokan","Tip;Waktu regular, Senin - Jumat;Kasual (conto...",21 days,2001 - 5000 pekerja,Retail/Merchandise,Job Description:Responsible for HSE implementa...,NaN,job description responsible for hse implementa...,"['operation management', 'analytical skill', '...",7
4,4,5,Japanese Interpreter,Bekasi,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Lainnya,Jurnalis/Editor",NaN,23 days,201 - 500 pekerja,Manajemen/Konsulting HR,Overview: Our clients is manufacture for autom...,NaN,overview our clients is manufacture for automo...,"['japanese', 'translator', 'english', 'non', '...",0


## Pengecekan Distribusi Kelas Target

Bagian ini bertujuan untuk memvalidasi kualitas dan karakteristik data target setelah proses pengelompokan serta transformasi label dilakukan.

- **`.fillna("unknown")`**  
  Digunakan sebagai langkah preventif untuk memastikan tidak terdapat nilai kosong (*NaN*) pada label target. Jika ditemukan, nilai tersebut akan digantikan dengan label **"unknown"**.
- **`y.nunique()`**  
  Menghitung jumlah total kategori unik (kelas) yang akan diprediksi oleh model. Informasi ini penting untuk menentukan konfigurasi *output layer*, khususnya jumlah neuron pada model *neural network*.
- **`y.value_counts()`**  
  Menampilkan distribusi frekuensi masing-masing kelas target. Analisis ini krusial untuk mendeteksi potensi **imbalanced data**, yaitu kondisi ketika jumlah data antar kelas tidak seimbang, yang dapat berdampak pada performa dan kemampuan generalisasi model, terutama pada kelas minoritas.

In [7]:
y = df["job_function_group"].fillna("unknown")
print("n_classes:", y.nunique())
print(y.value_counts())

n_classes: 10
job_function_group
9    5424
6    2895
3    1929
7    1638
4    1476
2    1302
5     767
0     714
8     599
1     438
Name: count, dtype: int64


## Downsampling untuk Penyeimbangan Kelas (*Undersampling*)

Bagian ini bertujuan untuk menangani permasalahan ketidakseimbangan data (*imbalanced data*) dengan cara membatasi jumlah sampel pada setiap kelas target, sehingga distribusi data menjadi lebih proporsional.

- **`MAX_PER_CLASS = 600`**  
  Menetapkan jumlah maksimum sampel untuk setiap kategori agar kelas dengan jumlah data besar tidak mendominasi proses pelatihan model.
- **`df.groupby(...)`**  
  Mengelompokkan dataset berdasarkan kategori fungsi pekerjaan yang telah didefinisikan sebelumnya.
- **`group.sample(...)`**  
  Jika suatu kategori memiliki jumlah data lebih dari batas yang ditentukan, sistem akan mengambil sebanyak 600 sampel secara acak. Parameter `random_state = 42` digunakan untuk memastikan proses sampling bersifat deterministik dan dapat direproduksi.
- **`pd.concat(dfs)`**  
  Menggabungkan kembali seluruh subset data hasil downsampling menjadi satu *DataFrame* utuh.
- **`reset_index(drop=True)`**  
  Mengatur ulang indeks baris agar berurutan kembali setelah proses penggabungan data.
**Tujuan Utama:**  
Mengurangi bias model terhadap kelas mayoritas, sehingga model tidak hanya optimal dalam memprediksi kategori dengan data melimpah, tetapi juga mampu mengenali pola pada kategori dengan jumlah data yang lebih sedikit secara lebih adil.

In [8]:
RANDOM_STATE = 42
MAX_PER_CLASS = 600

dfs = []

for label, group in df.groupby("job_function_group"):
    if len(group) > MAX_PER_CLASS:
        group = group.sample(
            n=MAX_PER_CLASS,
            random_state=RANDOM_STATE
        )
    dfs.append(group)

df = pd.concat(dfs).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5837 entries, 0 to 5836
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               5837 non-null   int64  
 1   id                       5837 non-null   int64  
 2   job_title                5837 non-null   object 
 3   location                 5837 non-null   object 
 4   salary_currency          5835 non-null   object 
 5   career_level             5837 non-null   object 
 6   experience_level         5837 non-null   object 
 7   education_level          5837 non-null   object 
 8   employment_type          5837 non-null   object 
 9   job_function             5837 non-null   object 
 10  job_benefits             4876 non-null   object 
 11  company_process_time     4531 non-null   object 
 12  company_size             5244 non-null   object 
 13  company_industry         5564 non-null   object 
 14  job_description         

## Penyatuan Fitur Kontekstual (Feature Engineering)

Tahap ini bertujuan memperkaya representasi data dengan menggabungkan beberapa fitur teks menjadi satu konteks utuh yang lebih informatif bagi model.

- **Konkatenasi String**  
  Menggabungkan tiga kolom utama, yaitu `job_description`, `experience_level`, dan `education_level`, menggunakan pemisah tanda hubung (`-`) untuk menjaga keterbacaan konteks.
- **`df["role_text"]`**  
  Membentuk kolom baru yang merepresentasikan deskripsi lengkap sebuah lowongan kerja.  
  Contoh:  
  *"Java Developer - 3–5 tahun - S1"*
- **Tujuan Semantik**  
  Penyatuan fitur ini membantu model NLP, seperti *Sentence Transformer*, memahami keterkaitan antara deskripsi pekerjaan, tingkat pengalaman, dan kualifikasi pendidikan dalam satu representasi vektor yang konsisten. Pendekatan ini umumnya meningkatkan kualitas embedding dan relevansi hasil pemodelan.

In [9]:
df["role_text"] = df["job_description"] + " - " + df['experience_level'] + " - " + df["education_level"]

In [10]:
df.head()

,Unnamed: 0,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary,job_description_cleaned,skills,job_function_group,role_text
0,2942,5894,Photographer,Jakarta Raya,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,Tidak terspesifikasi,Kontrak,"Seni/Media/Komunikasi,Seni / Desain Kreatif","Waktu regular, Senin - Jumat;Kasual (contoh: K...",24 days,501 - 1000 pekerja,Transportasi/Logistik,What is ShipperShipper is a growing technology...,8000000.0,what is shippershipper is a growing technology...,"['warehouse management', 'post production', 's...",0,What is ShipperShipper is a growing technology...
1,8387,16927,HEAD CREATIVE,Jakarta Raya,IDR,Supervisor/Koordinator,2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Seni/Media/Komunikasi,Seni / Desain Kreatif",Asuransi kesehatan;Bisnis (contoh: Kemeja),25 days,201 - 500 pekerja,Kesehatan/Medis,HEAD CREATIVEKualifikasiMemiliki 2 tahun penga...,NaN,head creativekualifikasimemiliki tahun pengala...,"['social medium', 'e commerce', 'target market...",0,HEAD CREATIVEKualifikasiMemiliki 2 tahun penga...
2,1123,2243,Japanese Interpreter,Jakarta Barat,IDR,Pegawai (non-manajemen & non-supervisor),3 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Lainnya,Jurnalis/Editor","Tip;Asuransi kesehatan;Waktu regular, Senin - ...",5 days,51 - 200 pekerja,Manajemen/Konsulting HR,JAPANESE INTERPRETER (TANGERANG) [50020]COMPAN...,NaN,japanese interpreter tangerang company categor...,"['japanese', 'electrical', 'communication', 'e...",0,JAPANESE INTERPRETER (TANGERANG) [50020]COMPAN...
3,7544,15212,Content Creator,Jakarta Barat,IDR,Pegawai (non-manajemen & non-supervisor),3 tahun,Sarjana (S1),Penuh Waktu,"Seni/Media/Komunikasi,Seni / Desain Kreatif","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",24 days,1- 50 pekerja,Makanan & Minuman/Katering/Restoran,CONTENT CREATORLocation : ...,5000000.0,content creatorlocation j...,"['digital marketing', 'social medium', 'adobe ...",0,CONTENT CREATORLocation : ...
4,16005,32356,Graphic Designer,Bali,IDR,Pegawai (non-manajemen & non-supervisor),1 tahun,Sarjana (S1),Penuh Waktu,"Seni/Media/Komunikasi,Seni / Desain Kreatif",NaN,NaN,NaN,Olahraga,Responsibilities :Taking design briefs to unde...,3600000.0,responsibilities taking design briefs to under...,"['design brief', 'graphic design', 'adobe phot...",0,Responsibilities :Taking design briefs to unde...


## Splitting Dataset dan Persiapan Label

Tahap ini bertujuan untuk membagi dataset ke dalam data latih dan data uji agar evaluasi model dapat dilakukan secara objektif dan terukur.

- **Konversi Tipe Data**  
  - **`X_text`**  
    Memastikan seluruh fitur teks bertipe `str` untuk menghindari error pada tahap *vectorization* atau *embedding*.  
  - **`y`**  
    Memastikan label target bertipe `int`, sesuai dengan kebutuhan komputasi *loss function* pada model klasifikasi.

- **`train_test_split`**  
  Digunakan untuk membagi data dengan proporsi **80% data latih** dan **20% data uji**.
- **`stratify=y`**  
  Parameter penting yang menjaga distribusi kelas pada data latih dan data uji tetap konsisten dengan dataset asli. Hal ini mencegah hilangnya representasi kelas tertentu pada salah satu himpunan data.
- **`data`**  
  Menyimpan hasil pembagian dataset dalam bentuk *tuple* agar lebih terstruktur dan mudah digunakan pada tahap pelatihan maupun evaluasi model.

In [11]:
X_text = df["role_text"].astype(str)
y = df["job_function_group"].astype(int)

num_classes = y.nunique()

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X_text,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

data = (X_train, y_train), (X_test, y_test)

# Model for Predict Job Function

## Ekstraksi Fitur dan Seleksi Fitur (N-Gram Vectorization)

Tahap ini bertujuan untuk mengubah teks mentah menjadi representasi numerik yang dapat diproses oleh model *Machine Learning* dengan mengombinasikan metode **TF-IDF** dan **Anova F-test**.

- **`TfidfVectorizer` (N-Gram & TF-IDF)**  
  Digunakan untuk membangun representasi fitur berbasis bobot kata.
  - **`ngram_range=(1, 2)`**  
    Mengambil kata tunggal (*unigram*) dan pasangan kata (*bigram*) untuk menangkap konteks yang lebih kaya, misalnya *"data"* dan *"data scientist"*.
  - **`min_df=2`**  
    Mengabaikan kata yang hanya muncul pada satu dokumen guna mengurangi noise dan mencegah *overfitting*.
  - **`fit_transform()`**  
    Mempelajari kosakata dari data latih sekaligus mengonversinya menjadi bobot TF-IDF.

- **`SelectKBest` (Seleksi Fitur)**  
  Digunakan untuk memilih fitur yang paling informatif.
  - Menggunakan uji statistik **`f_classif`** (ANOVA F-test) untuk menyeleksi **20.000 fitur terbaik** yang memiliki korelasi tertinggi dengan label target.  
    Langkah ini bertujuan mengurangi fitur yang tidak relevan serta mempercepat proses pelatihan model.
- **`.toarray()`**  
  Mengonversi hasil vektorisasi dari format *sparse matrix* menjadi *dense array* NumPy agar kompatibel dengan input *neural network*.

In [13]:
NGRAM_RANGE = (1, 2)
TOP_K = 20000
TOKEN_MODE = 'word'
MIN_DOCUMENT_FREQUENCY = 2

def ngram_vectorize(train_texts, train_labels, val_texts):
    kwargs = {
        'ngram_range': NGRAM_RANGE,
        'dtype': 'float32',
        'strip_accents': 'unicode',
        'decode_error': 'replace',
        'analyzer': TOKEN_MODE,
        'min_df': MIN_DOCUMENT_FREQUENCY
    }
    vectorizer = TfidfVectorizer(**kwargs)

    X_train = vectorizer.fit_transform(train_texts)
    X_val = vectorizer.transform(val_texts)

    selector = SelectKBest(f_classif, k=min(TOP_K, X_train.shape[1]))
    selector.fit(X_train, train_labels)
    X_train = selector.transform(X_train)
    X_val = selector.transform(X_val)

    X_train = X_train.toarray()
    X_val = X_val.toarray()
    
    return X_train, X_val, vectorizer, selector

## Konfigurasi Output Layer Model

Bagian ini menjelaskan fungsi pembantu yang digunakan untuk menentukan konfigurasi *output layer* secara otomatis, berdasarkan jenis masalah klasifikasi yang ditangani oleh model.

- **Klasifikasi Biner (`num_classes == 2`)**  
  - **Jumlah Unit = 1**  
    Hanya diperlukan satu neuron pada lapisan output untuk memprediksi probabilitas antara dua kelas.  
  - **Fungsi Aktivasi = `sigmoid`**  
    Menghasilkan nilai probabilitas dalam rentang 0 hingga 1, yang merepresentasikan peluang suatu data termasuk ke dalam kelas positif.

- **Klasifikasi Multi-kelas (`num_classes > 2`)**  
  - **Jumlah Unit = `num_classes`**  
    Setiap kategori direpresentasikan oleh satu neuron pada lapisan output.  
  - **Fungsi Aktivasi = `softmax`**  
    Menghasilkan distribusi probabilitas antar kelas, di mana total probabilitas seluruh neuron bernilai 1. Kelas dengan nilai probabilitas tertinggi dipilih sebagai hasil prediksi model.

In [14]:
def _get_last_layer_units_and_activation(num_classes):
    if num_classes == 2:
        activation = 'sigmoid'
        units = 1
    else:
        activation = 'softmax'
        units = num_classes
    return units, activation

## Pembangunan Model MLP (Multi-Layer Perceptron)

Bagian ini menjelaskan fungsi yang digunakan untuk membangun arsitektur *Neural Network* sederhana berbasis MLP, yang umum diterapkan pada data teks hasil vektorisasi, seperti TF-IDF.

- **`models.Sequential()`**  
  Menginisialisasi model *Sequential*, di mana lapisan-lapisan jaringan saraf disusun secara berurutan dari lapisan input hingga lapisan output.
- **Lapisan Input dan Regularisasi**  
  - **`layers.Input`**  
    Menentukan dimensi data masukan sesuai dengan jumlah fitur yang digunakan, misalnya 20.000 fitur hasil seleksi TF-IDF.  
  - **`layers.Dropout`**  
    Berfungsi sebagai mekanisme regularisasi untuk mengurangi risiko *overfitting* dengan menonaktifkan neuron secara acak selama proses pelatihan.

- **Hidden Layers (Lapisan Tersembunyi)**  
  - **`layers.Dense`**  
    Merepresentasikan lapisan saraf *fully connected* yang memproses representasi fitur secara menyeluruh.  
  - **`activation='relu'`**  
    Menggunakan fungsi aktivasi ReLU untuk menambahkan sifat non-linear, sehingga model mampu mempelajari pola data yang lebih kompleks.

- **Lapisan Output**  
  Lapisan terakhir dikonfigurasi melalui fungsi `_get_last_layer_units_and_activation`, yang secara otomatis menentukan jumlah neuron dan fungsi aktivasi yang sesuai, yaitu **Sigmoid** untuk klasifikasi biner atau **Softmax** untuk klasifikasi multi-kelas.

In [15]:
def mlp_model(num_layers, units, dropout_rate, input_shape, num_classes):
    op_units, op_activation = _get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dropout(rate=dropout_rate))

    for _ in range(num_layers-1):
        model.add(layers.Dense(units=units, activation='relu'))
        model.add(layers.Dropout(rate=dropout_rate))

    model.add(layers.Dense(units=op_units, activation=op_activation))
    return model

## Eksekusi Vektorisasi dan Persistensi Preprocessor

Tahap ini menerapkan transformasi teks menjadi fitur numerik serta menyimpan komponen prapemrosesan agar dapat digunakan kembali pada tahap selanjutnya, seperti validasi lanjutan atau *deployment*.

**Penerapan `ngram_vectorize`**  
- Mengubah data teks `train_texts` dan `val_texts` menjadi representasi numerik (**X_train** dan **X_val**) berdasarkan bobot TF-IDF yang telah melalui proses seleksi fitur.  
- **Catatan Penting:** Objek `vectorizer` dan `selector` hanya dipelajari (*fit*) menggunakan data latih untuk mencegah kebocoran data (*data leakage*) yang dapat menyebabkan evaluasi model menjadi bias.

**Penyimpanan Objek (`joblib.dump`)**  
- Menyimpan `vectorizer` (kosakata dan bobot TF-IDF) serta `selector` (indeks fitur terpilih) ke dalam satu berkas bernama `text_preprocessor.joblib`.  
- **Tujuan:** Menjamin bahwa data baru di masa mendatang diproses dengan skema yang konsisten dan identik—menggunakan kosakata serta fitur yang sama—sebelum diteruskan ke model untuk proses inferensi.

In [16]:
(train_texts, train_labels), (val_texts, val_labels) = data
X_train, X_val, vectorizer, selector = ngram_vectorize(train_texts, train_labels, val_texts)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:2043: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. float32 'dtype' will be converted to np.float64.
  warnings.warn(


In [17]:
joblib.dump(
    {"vectorizer": vectorizer, "selector": selector},
    "text_preprocessor.joblib"
)

['text_preprocessor.joblib']

## Fungsi Pelatihan Model (Training Loop)

Bagian ini mengatur seluruh alur pelatihan model, mulai dari inisialisasi arsitektur hingga penyimpanan model akhir yang siap digunakan.

- **Inisialisasi Model**  
  Model dibangun dengan memanggil fungsi `mlp_model` menggunakan parameter yang dapat disesuaikan (*hyperparameters*), seperti jumlah lapisan (`num_layers`), jumlah unit pada setiap lapisan, dan *dropout rate*.
- **Konfigurasi Loss dan Optimizer**  
  - **Loss Function**: Menggunakan `sparse_categorical_crossentropy` karena label target direpresentasikan dalam bentuk bilangan bulat dan termasuk kasus klasifikasi multi-kelas.  
  - **Optimizer (Adam)**: Algoritma optimasi adaptif yang digunakan untuk memperbarui bobot model secara efisien berdasarkan nilai `learning_rate`.
- **Early Stopping (Callback)**  
  Mekanisme ini digunakan untuk mencegah *overfitting*. Proses pelatihan akan dihentikan secara otomatis apabila nilai `val_loss` tidak menunjukkan perbaikan selama dua *epoch* berturut-turut (`patience=2`).
- **Proses Pelatihan (Model Fit)**  
  Model dilatih menggunakan data `X_train`. Performa model pada setiap *epoch* dicatat dan disimpan dalam objek `history` untuk keperluan evaluasi dan analisis lebih lanjut.
- **Penyimpanan Model**  
  `model.save('mlp_model.keras')` digunakan untuk menyimpan arsitektur, bobot, serta konfigurasi pelatihan ke dalam satu berkas `.keras`. Dengan demikian, model dapat langsung digunakan kembali untuk inferensi tanpa perlu dilatih ulang.

In [18]:
def train_ngram_model(
    data,
    learning_rate=1e-3,
    epochs=1000,
    batch_size=128,
    num_layers=2,
    units=64,
    dropout_rate=0.2
):

    model = mlp_model(
        num_layers=num_layers,
        units=units,
        dropout_rate=dropout_rate,
        input_shape=X_train.shape[1:],
        num_classes=num_classes
    )

    if num_classes == 2:
        loss = 'binary_crossentropy'
    else:
        loss = 'sparse_categorical_crossentropy'
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=2
    )]

    history = model.fit(
        X_train,
        train_labels,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=(X_val, val_labels),
        verbose=2,
        batch_size=batch_size
    )

    history = history.history
    print('Validation accuracy: {acc}, loss: {loss}'.format(
            acc=history['val_acc'][-1], loss=history['val_loss'][-1]))

    model.save('mlp_model.keras')
    return history['val_acc'][-1], history['val_loss'][-1]

## Eksekusi Pelatihan Model MLP

Bagian ini menjalankan proses pelatihan model *Machine Learning* menggunakan arsitektur MLP dengan konfigurasi *hyperparameter* yang telah ditentukan pada dataset yang sudah dipraproses.

- **Parameter yang Digunakan**
  - **`learning_rate = 1e-3`**  
    Menentukan kecepatan pembaruan bobot model. Nilai 0,001 merupakan konfigurasi umum yang stabil untuk optimizer Adam.
  
  - **`epochs = 1000`**  
    Menetapkan jumlah maksimum iterasi pelatihan. Dalam praktiknya, pelatihan biasanya berhenti lebih awal karena mekanisme *Early Stopping* ketika performa validasi tidak lagi meningkat.
  
  - **`batch_size = 128`**  
    Menentukan jumlah sampel yang diproses dalam satu langkah pembaruan bobot.
  
  - **`num_layers = 2` dan `units = 64`**  
    Membangun arsitektur jaringan dengan dua *hidden layer*, masing-masing terdiri dari 64 neuron.

- **Proses yang Terjadi**
  1. Model menerima input berupa fitur teks hasil vektorisasi TF-IDF.
  2. Model melakukan *forward pass* untuk menghasilkan prediksi, kemudian *backward pass* untuk memperbarui bobot berdasarkan nilai kesalahan.
  3. Pada akhir setiap *epoch*, performa model dievaluasi menggunakan data validasi (`X_val`) untuk memantau indikasi *overfitting*.

- **Output Akhir**  
  Setelah pelatihan selesai, sistem menampilkan nilai *loss* dan akurasi terakhir pada data validasi, lalu menyimpan model terlatih ke dalam file `mlp_model.keras` untuk digunakan pada tahap inferensi atau *deployment*.

In [19]:
train_ngram_model(data, learning_rate=1e-3, epochs=1000, batch_size=128, num_layers=2, units=64, dropout_rate=0.2)

Epoch 1/1000
37/37 - 1s - 25ms/step - acc: 0.5160 - loss: 2.2172 - val_acc: 0.6764 - val_loss: 2.0872
Epoch 2/1000
37/37 - 0s - 10ms/step - acc: 0.7271 - loss: 1.9150 - val_acc: 0.7432 - val_loss: 1.7725
Epoch 3/1000
37/37 - 0s - 11ms/step - acc: 0.7777 - loss: 1.5516 - val_acc: 0.7705 - val_loss: 1.4490
Epoch 4/1000
37/37 - 0s - 10ms/step - acc: 0.7965 - loss: 1.2345 - val_acc: 0.7740 - val_loss: 1.1942
Epoch 5/1000
37/37 - 0s - 10ms/step - acc: 0.8139 - loss: 0.9989 - val_acc: 0.7851 - val_loss: 1.0214
Epoch 6/1000
37/37 - 0s - 11ms/step - acc: 0.8344 - loss: 0.8396 - val_acc: 0.7868 - val_loss: 0.9090
Epoch 7/1000
37/37 - 1s - 14ms/step - acc: 0.8477 - loss: 0.7221 - val_acc: 0.7962 - val_loss: 0.8334
Epoch 8/1000
37/37 - 0s - 12ms/step - acc: 0.8631 - loss: 0.6370 - val_acc: 0.7997 - val_loss: 0.7807
Epoch 9/1000
37/37 - 0s - 11ms/step - acc: 0.8711 - loss: 0.5713 - val_acc: 0.7971 - val_loss: 0.7426
Epoch 10/1000
37/37 - 0s - 10ms/step - acc: 0.8758 - loss: 0.5192 - val_acc: 0.801

(0.8082191944122314, 0.6349846124649048)

## Fungsi Prediksi Model (Inference)

Bagian ini mendefinisikan alur inferensi untuk mengubah teks mentah menjadi prediksi kategori fungsi pekerjaan menggunakan model yang telah dilatih.

- **`models.load_model(...)`**  
  Memuat kembali model *neural network* yang sebelumnya telah disimpan. Dengan pendekatan ini, proses prediksi dapat dijalankan di sesi atau lingkungan berbeda tanpa perlu melakukan pelatihan ulang.

- **Pipeline Transformasi Data**  
  - **`vectorizer.transform(texts)`**  
    Mengonversi teks baru ke representasi TF-IDF menggunakan kosakata yang dipelajari pada tahap pelatihan.  
  - **`selector.transform(X)`**  
    Menerapkan kembali seleksi fitur untuk mempertahankan hanya fitur-fitur terbaik yang sama seperti saat pelatihan, kemudian mengubahnya ke dalam format *dense array*.

- **Proses Prediksi**  
  - **`model.predict(X)`**  
    Menghasilkan distribusi probabilitas untuk setiap kategori target (misalnya: Finance, IT, HR).  
  - **`probs.argmax(axis=1)`**  
    Menentukan label prediksi akhir dengan memilih indeks kelas yang memiliki probabilitas tertinggi.

In [20]:
model = models.load_model('mlp_model.keras')

def predict_job_function(texts, vectorizer, selector, model):
    X = vectorizer.transform(texts)
    X = selector.transform(X).toarray()
    probs = model.predict(X)
    labels = probs.argmax(axis=1)
    return labels, probs

## Definisi Data Pengujian (Persona Berbasis Teks)

Bagian ini mendefinisikan sepuluh variabel *persona* yang merepresentasikan profil profesional dari berbagai bidang industri. Kumpulan data ini digunakan sebagai **data pengujian (test case)** untuk mengevaluasi kemampuan model dalam melakukan klasifikasi teks.

- **Tujuan**  
  Menyediakan input berupa teks naratif yang panjang dan kaya konteks, dengan kata kunci khas tiap industri (misalnya *“pencatatan transaksi”* untuk Finance, *“silabus”* untuk Education, atau *“backend”* untuk IT), sehingga model diuji pada skenario yang mendekati penggunaan nyata.

- **Struktur Data**  
  Setiap persona disimpan dalam bentuk *list* yang berisi satu atau lebih string deskriptif, mencakup:
  - Pengalaman kerja.
  - Tugas dan tanggung jawab harian (*daily tasks*).
  - Keahlian teknis (*hard skills*).
  - Minat dan fokus profesional.

- **Peran dalam Alur Kerja**  
  Teks persona ini akan dimasukkan ke dalam fungsi `predict_job_function` untuk memverifikasi apakah model mampu mengklasifikasikan deskripsi profil yang kompleks dan detail ke dalam kategori fungsi pekerjaan yang sesuai (misalnya `persona_6_it_software` diprediksi sebagai label **IT & Software**).

In [21]:
persona_0_creative_media = ["""Saya memiliki pengalaman panjang di bidang kreatif dan media, dengan fokus utama pada produksi konten visual dan pengelolaan komunikasi digital. 
Selama beberapa tahun terakhir, saya terbiasa bekerja dengan berbagai format konten seperti desain grafis, video pendek, fotografi, serta penulisan 
konten untuk media sosial dan platform digital lainnya. Saya memahami bagaimana sebuah pesan dapat disampaikan secara efektif melalui visual dan 
narasi yang kuat, sehingga audiens dapat terhubung secara emosional dengan brand atau produk yang ditampilkan. Dalam pekerjaan sehari-hari, saya 
sering terlibat dalam proses brainstorming ide kreatif bersama tim, mulai dari tahap konsep hingga eksekusi akhir. Saya terbiasa mengembangkan 
konsep kampanye, menentukan tone komunikasi, serta memastikan konsistensi identitas visual di berbagai kanal. Selain itu, saya juga terbiasa 
bekerja dengan deadline yang ketat dan melakukan revisi berdasarkan masukan dari stakeholder atau klien. Saya memiliki ketertarikan yang besar 
terhadap tren media digital, perkembangan platform sosial, dan perubahan perilaku audiens. Hal ini mendorong saya untuk terus belajar dan 
menyesuaikan gaya konten agar tetap relevan. Saya terbiasa menggunakan berbagai tools desain dan editing, serta memahami dasar analisis performa 
konten untuk melihat efektivitas kampanye yang dijalankan. Bagi saya, dunia kreatif bukan hanya soal estetika, tetapi juga tentang bagaimana sebuah
ide dapat dikomunikasikan dengan jelas dan berdampak. Saya menikmati proses eksplorasi ide, eksperimen visual, dan kolaborasi lintas fungsi dengan 
tim marketing maupun komunikasi. Saya tertarik untuk terus berkembang di bidang creative dan media, khususnya dalam peran yang memungkinkan saya 
berkontribusi pada pengembangan brand dan storytelling yang kuat.
"""]

persona_1_education = ["""
Saya memiliki latar belakang dan pengalaman panjang di bidang pendidikan, baik dalam konteks pengajaran langsung maupun pengembangan materi 
pembelajaran. Saya terbiasa berinteraksi dengan peserta didik dari berbagai latar belakang, memahami kebutuhan belajar mereka, serta menyesuaikan 
metode pengajaran agar materi dapat dipahami dengan lebih efektif. Saya percaya bahwa proses belajar tidak hanya tentang penyampaian materi, tetapi 
juga tentang membangun pemahaman, motivasi, dan rasa ingin tahu. Dalam peran saya sebagai pendidik, saya sering terlibat dalam penyusunan silabus, 
perencanaan pembelajaran, serta evaluasi hasil belajar. Saya terbiasa menggunakan berbagai pendekatan, baik konvensional maupun berbasis teknologi, 
untuk mendukung proses belajar mengajar. Saya juga memiliki pengalaman dalam melakukan pendampingan dan bimbingan kepada peserta didik yang 
membutuhkan perhatian khusus. Saya memiliki ketertarikan pada pengembangan sistem pendidikan, inovasi metode pembelajaran, serta pemanfaatan 
teknologi untuk meningkatkan kualitas pendidikan. Saya percaya bahwa pendidikan memiliki peran penting dalam membentuk karakter dan kompetensi 
seseorang. Oleh karena itu, saya selalu berusaha menciptakan lingkungan belajar yang kondusif, inklusif, dan mendorong partisipasi aktif. Selain 
mengajar, saya juga terbiasa bekerja secara administratif di lingkungan pendidikan, seperti pengelolaan data akademik dan koordinasi kegiatan 
pembelajaran. Saya memiliki kemampuan komunikasi yang baik, sabar, dan terorganisir. Saya tertarik untuk terus berkontribusi di dunia pendidikan,
baik sebagai pengajar, pengembang kurikulum, maupun peran lain yang mendukung proses belajar.
"""]

persona_2_engineering = ["""
Saya memiliki latar belakang di bidang engineering dan pengalaman bekerja dalam lingkungan teknis yang menuntut ketelitian serta pemahaman sistem
yang kuat. Saya terbiasa menganalisis permasalahan teknis, merancang solusi, serta memastikan implementasi berjalan sesuai dengan spesifikasi dan 
standar yang telah ditetapkan. Dalam pekerjaan saya, aspek keselamatan, efisiensi, dan keandalan selalu menjadi prioritas utama. Saya sering bekerja 
dengan dokumentasi teknis, perhitungan, serta koordinasi dengan berbagai pihak seperti tim operasional dan manajemen proyek. Saya memahami bahwa 
sebuah solusi engineering tidak hanya harus bekerja secara teknis, tetapi juga harus dapat diterapkan secara praktis dan berkelanjutan. Oleh karena
itu, saya terbiasa mempertimbangkan berbagai faktor seperti biaya, waktu, dan risiko dalam setiap keputusan teknis. Saya menikmati proses pemecahan
masalah dan tantangan teknis yang kompleks. Saya terbiasa bekerja secara sistematis dan terstruktur, serta mampu bekerja baik secara mandiri maupun 
dalam tim. Saya juga memiliki ketertarikan untuk terus mempelajari teknologi dan metode baru yang dapat meningkatkan kualitas solusi engineering 
yang saya kerjakan. Bagi saya, bidang engineering adalah tentang mengubah konsep dan teori menjadi solusi nyata yang dapat memberikan manfaat. 
Saya tertarik untuk terus berkembang dalam peran engineering yang memungkinkan saya berkontribusi pada pengembangan sistem, infrastruktur, atau
produk yang andal dan efisien.
"""]

persona_3_finance_accounting = ["""
Saya memiliki pengalaman panjang di bidang keuangan dan akuntansi, dengan fokus pada pengelolaan data keuangan yang akurat dan transparan. 
Saya terbiasa menangani pencatatan transaksi, penyusunan laporan keuangan, serta analisis anggaran. Ketelitian dan konsistensi adalah hal yang 
sangat saya jaga dalam bekerja dengan angka dan data finansial. Dalam keseharian, saya sering bekerja dengan laporan keuangan bulanan dan tahunan,
melakukan rekonsiliasi, serta memastikan kepatuhan terhadap standar akuntansi. Saya memahami pentingnya data keuangan sebagai dasar pengambilan 
keputusan bisnis, sehingga saya selalu berusaha menyajikan informasi yang jelas dan dapat dipercaya. Saya juga terbiasa bekerja dengan berbagai 
pihak, termasuk tim internal dan auditor, untuk memastikan proses keuangan berjalan dengan baik. Saya memiliki kemampuan analisis yang kuat dan 
mampu mengidentifikasi potensi risiko maupun peluang dari sisi keuangan. Selain itu, saya terbiasa menggunakan tools seperti spreadsheet dan sistem
akuntansi untuk mendukung pekerjaan. Saya tertarik untuk terus berkembang di bidang finance dan accounting, khususnya dalam peran yang
memungkinkan saya berkontribusi pada stabilitas dan pertumbuhan organisasi melalui pengelolaan keuangan yang baik.
"""]

persona_4_hr_administration = ["""
Saya memiliki pengalaman di bidang human resources dan administrasi, dengan fokus pada pengelolaan sumber daya manusia dan proses administratif 
perusahaan. Saya terbiasa menangani data karyawan, administrasi kepegawaian, serta mendukung proses rekrutmen dan onboarding. Saya memahami
pentingnya ketertiban administrasi dalam mendukung operasional organisasi. Dalam peran saya, saya sering menjadi penghubung antara karyawan dan 
manajemen, memastikan informasi tersampaikan dengan jelas dan tepat. Saya memiliki kemampuan komunikasi yang baik dan terbiasa menangani dokumen 
serta data dengan tingkat kerahasiaan yang tinggi. Saya juga terbiasa bekerja dengan prosedur dan kebijakan perusahaan. Saya tertarik pada 
pengembangan sistem HR yang efisien dan berorientasi pada kesejahteraan karyawan. Saya percaya bahwa pengelolaan sumber daya manusia yang baik 
dapat menciptakan lingkungan kerja yang produktif dan harmonis. Oleh karena itu, saya selalu berusaha bekerja secara rapi, terstruktur, dan
profesional dalam setiap tugas administrasi yang saya tangani.
"""]

persona_5_hospitality_service = ["""
Saya memiliki pengalaman panjang di bidang hospitality dan layanan pelanggan, dengan fokus pada pemberian pelayanan yang ramah dan profesional. 
Saya terbiasa berinteraksi langsung dengan pelanggan, memahami kebutuhan mereka, serta memastikan pengalaman yang positif selama menggunakan 
layanan. Saya memahami bahwa kepuasan pelanggan merupakan faktor utama dalam industri jasa. Saya sering bekerja dalam lingkungan yang dinamis dan 
menuntut kecepatan serta ketepatan. Saya terbiasa menangani keluhan pelanggan dengan sikap tenang dan solutif. Selain itu, saya juga memahami 
pentingnya kerja sama tim dalam memastikan kelancaran operasional layanan. Saya memiliki kemampuan komunikasi interpersonal yang baik dan mampu 
bekerja dalam sistem shift. Saya menikmati pekerjaan yang melibatkan interaksi dengan banyak orang dan memberikan solusi secara langsung. 
Saya tertarik untuk terus berkembang di bidang hospitality dan service, khususnya dalam peran yang menekankan kualitas layanan dan kepuasan
pelanggan.
"""]

persona_6_it_software = ["""
Saya memiliki latar belakang dan pengalaman di bidang teknologi informasi dan pengembangan perangkat lunak. Saya terbiasa bekerja dengan sistem
berbasis software, memahami alur pengembangan aplikasi, serta memecahkan masalah teknis menggunakan pendekatan logis. Saya memiliki ketertarikan
besar terhadap teknologi dan bagaimana teknologi dapat digunakan untuk meningkatkan efisiensi kerja. Saya sering bekerja dengan data, aplikasi,
dan sistem backend, serta memahami konsep database dan integrasi sistem. Saya terbiasa bekerja secara terstruktur dan mengikuti proses pengembangan 
yang sistematis. Selain itu, saya juga mampu bekerja dalam tim dan berkomunikasi dengan pihak non-teknis untuk menjelaskan solusi yang dikembangkan.
Saya tertarik untuk terus belajar dan mengikuti perkembangan teknologi terbaru. Bagi saya, bidang IT dan software adalah ruang yang terus 
berkembang dan menawarkan tantangan intelektual yang menarik. Saya ingin terus berkontribusi dalam pengembangan solusi digital yang bermanfaat
dan scalable.
"""]

persona_7_manufacturing_operations = ["""
Saya memiliki pengalaman di bidang manufaktur dan operasional, dengan fokus pada pengawasan proses produksi dan efisiensi kerja. 
Saya terbiasa bekerja di lingkungan yang terstruktur dengan prosedur yang jelas. Saya memahami pentingnya kualitas produk, keselamatan kerja, dan 
kelancaran operasional dalam proses manufaktur. Saya sering terlibat dalam pemantauan proses produksi, koordinasi dengan tim, serta memastikan
target operasional tercapai. Saya terbiasa bekerja dengan jadwal dan target yang ketat, serta memahami pentingnya kerja sama tim dalam 
lingkungan produksi. Saya tertarik pada peningkatan efisiensi proses dan optimalisasi operasional. Saya percaya bahwa perbaikan kecil yang 
konsisten dapat memberikan dampak besar pada produktivitas. Saya ingin terus berkembang di bidang manufacturing dan operations dengan fokus 
pada proses yang efektif dan berkelanjutan.
"""]

persona_8_other = ["""
Saya memiliki latar belakang pekerjaan yang bersifat umum dan lintas fungsi. Saya terbiasa menangani berbagai tugas yang beragam, mulai dari
administrasi hingga dukungan operasional. Saya fleksibel dan mudah beradaptasi dengan lingkungan kerja yang berubah. Saya memiliki kemampuan
belajar yang cepat dan siap mendukung berbagai kebutuhan organisasi. Saya terbiasa bekerja dengan berbagai tim dan memahami pentingnya komunikasi 
yang jelas. Saya tertarik pada peran yang memungkinkan saya berkontribusi secara luas dan mendukung kelancaran operasional perusahaan.
"""]

persona_9_sales_marketing = ["""
Saya memiliki pengalaman di bidang sales dan marketing, dengan fokus pada pengembangan pasar dan pencapaian target penjualan. Saya terbiasa 
membangun hubungan dengan pelanggan, memahami kebutuhan mereka, serta menawarkan solusi yang sesuai. Saya memiliki kemampuan komunikasi dan 
negosiasi yang baik. Saya sering terlibat dalam perencanaan strategi pemasaran, pelaksanaan kampanye, serta evaluasi hasil penjualan. Saya memahami
pentingnya analisis pasar dan perilaku pelanggan dalam menentukan strategi yang efektif. Saya tertarik untuk terus berkembang di bidang sales dan 
marketing dengan fokus pada pertumbuhan bisnis dan kepuasan pelanggan.
"""]

## Uji Coba Prediksi Model dengan Data Baru

Bagian ini mendemonstrasikan proses pengujian model menggunakan teks input baru untuk mengevaluasi bagaimana model memberikan prediksi kategori.

- **`persona_random`**  
  Sebuah teks uji yang didefinisikan secara bebas (bertopik kucing) sebagai eksperimen tambahan. Namun, pada implementasi yang dijalankan, input yang digunakan adalah `persona_9_sales_marketing`.

- **`predict_job_function(...)`**  
  Fungsi ini menjalankan seluruh *inference pipeline*, mulai dari transformasi teks menjadi vektor TF-IDF, seleksi fitur yang relevan, hingga menghasilkan prediksi menggunakan model MLP.

- **`labels`**  
  Berisi indeks numerik dari kategori dengan probabilitas tertinggi. Pada kasus `persona_9_sales_marketing`, nilai ini diharapkan merujuk pada label **Sales & Marketing**.

- **`probs`**  
  Menampilkan distribusi probabilitas untuk seluruh kelas target. Informasi ini berguna untuk menilai tingkat kepercayaan model terhadap hasil prediksi (misalnya, probabilitas yang mendekati 1.0 menunjukkan keyakinan yang tinggi).

---

**Catatan:**  
Untuk menguji prediksi menggunakan teks tentang kucing, argumen fungsi dapat diganti menjadi:

```python
labels, probs = predict_job_function(persona_random, vectorizer, selector, model)

In [22]:
persona_random = ["""
Saya adalah seekor kucing dengan kemampuan untuk mencuri makanan, tidur siang panjang, lapar setiap 5 menit, beraktivitas setiap hari, nyakar jok motor
, bercita-cita untuk bisa menjadi sales. Saya menganggap manusia adalah babu yang perlu melayani saya, menggaruk leher saya ketika gatal. Visi saya adalah
untuk menguasasi negara, menanam sawit di kalimantan dan papua."""]

labels, probs = predict_job_function(persona_9_sales_marketing, vectorizer, selector, model)
print(labels)
print(probs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
[9]
[[0.01468597 0.00699696 0.00632274 0.01644747 0.01159591 0.00936187
  0.0049381  0.00587713 0.01246064 0.91131324]]


# Model untuk Get Recommendation based on Semantic Matching

## Pembuatan Semantic Embeddings (Vectorization)

Berbeda dengan pendekatan TF-IDF yang mengandalkan frekuensi kemunculan kata, tahap ini menggunakan metode *deep learning* untuk menangkap makna kontekstual dari teks secara lebih mendalam.

- **`SentenceTransformer`**  
  Memuat model **MiniLM-L12-v2** yang mendukung multibahasa, termasuk Bahasa Indonesia. Model ini mengonversi kalimat atau dokumen menjadi vektor numerik (*embeddings*) dalam ruang berdimensi tinggi.

- **Representasi Semantik**  
  Teks dengan makna yang serupa, seperti *“tenaga penjual”* dan *“sales”*, akan direpresentasikan oleh vektor yang saling berdekatan meskipun memiliki perbedaan secara leksikal.

- **`embed_texts`**  
  Fungsi ini memproses teks secara bertahap (*batching*) untuk menjaga efisiensi penggunaan memori, lalu mengembalikan hasil embedding dalam bentuk array NumPy.

- **Feature Engineering Berbasis Konteks**  
  - **`role_embeddings`**: Merepresentasikan konteks pekerjaan yang mencakup deskripsi peran, tingkat pengalaman, dan latar belakang pendidikan.  
  - **`skill_embeddings`**: Merepresentasikan kemampuan teknis spesifik yang diekstraksi dari kolom `skills`.

- **Output Shape**  
  Proses ini menghasilkan sebuah matriks embedding dengan ukuran `(jumlah

In [23]:
EMBED_MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"
embed_model = SentenceTransformer(EMBED_MODEL_NAME)

def embed_texts(texts, batch_size=64):
    return embed_model.encode(texts, show_progress_bar=True, batch_size=batch_size, convert_to_numpy=True)

role_texts = df["role_text"].tolist()
role_embeddings = embed_texts(role_texts)
print("role_embeddings", role_embeddings.shape)

Batches:   0%|          | 0/92 [00:00<?, ?it/s]

role_embeddings (5837, 384)


## Penyimpanan dan Indexing Vektor dengan FAISS

Bagian ini membangun sistem pencarian berbasis kemiripan (*similarity search*) yang memungkinkan pencarian data paling relevan berdasarkan kedekatan vektor secara efisien.

- **`faiss.IndexFlatL2(d)`**  
  Membuat indeks FAISS menggunakan metrik jarak Euclidean (L2). Parameter `d` merepresentasikan dimensi vektor embedding (misalnya 384 atau 768, tergantung model *transformer* yang digunakan). Tipe indeks *Flat* berarti seluruh vektor dibandingkan secara langsung (*brute-force*), sehingga hasil pencarian bersifat akurat.

- **`index.add(role_embeddings)`**  
  Menambahkan seluruh vektor embedding deskripsi pekerjaan ke dalam indeks FAISS. Setelah proses ini selesai, indeks siap menerima kueri dan melakukan pencarian kemiripan dalam waktu sangat singkat.

- **`index.ntotal`**  
  Menampilkan jumlah total vektor yang tersimpan di dalam indeks sebagai bentuk verifikasi bahwa seluruh data telah berhasil dimuat.

- **`faiss.write_index(...)`**  
  Menyimpan struktur indeks FAISS ke dalam file berekstensi `.idx`. Penyimpanan ini memungkinkan indeks digunakan kembali tanpa perlu membangun ulang setiap kali aplikasi dijalankan.

- **`np.save(...)`**  
  Menyimpan matriks embedding (baik *role embeddings* maupun *skill embeddings*) ke dalam format biner `.npy` milik NumPy. File ini berfungsi sebagai arsip vektor mentah yang dapat dimanfaatkan untuk analisis lanjutan atau kebutuhan komputasi lainnya.

In [24]:
d = role_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(role_embeddings)
print("index ntotal:", index.ntotal)

faiss.write_index(index, "faiss_role_index.idx")
np.save("role_embeddings.npy", role_embeddings)

index ntotal: 5837


## Penyusunan dan Penyimpanan Metadata Pekerjaan

Tahap ini bertujuan untuk mengaitkan hasil pencarian berbasis vektor (berupa indeks numerik) dengan informasi pekerjaan yang dapat dipahami dan ditampilkan kepada pengguna.

- **Seleksi Kolom**  
  Memilih kolom-kolom utama seperti `job_title`, `skills`, dan `job_description`. Informasi ini akan digunakan sebagai konten yang ditampilkan pada hasil pencarian atau sistem rekomendasi.

- **`to_dict(orient="records")`**  
  Mengonversi DataFrame menjadi daftar *dictionary* (*list of dict*), di mana setiap elemen merepresentasikan satu entri pekerjaan. Struktur ini memudahkan akses data berdasarkan indeks.

- **`pickle.dump`**  
  Menyimpan objek metadata ke dalam file biner (`.bin`) agar dapat dimuat kembali dengan cepat tanpa perlu pemrosesan ulang.

- **Konsistensi Urutan Data (Penting)**  
  Urutan data pada `metadata` **harus identik** dengan urutan vektor yang tersimpan di indeks **FAISS**. Dengan demikian, ketika FAISS mengembalikan indeks tertentu (misalnya indeks ke-5), sistem dapat langsung mengambil informasi pekerjaan yang sesuai melalui `metadata[5]`.


In [25]:
metadata = df[[
    "id", "job_title", "job_function", "skills", "job_description"
]].to_dict(orient="records")

In [26]:
with open("job_metadata.bin", "wb") as f:
    pickle.dump(metadata, f)

## Fungsi Pencarian Kemiripan (Similarity Search)

Fungsi ini merupakan komponen inti dalam sistem rekomendasi atau mesin pencari berbasis vektor. Tugas utamanya adalah menemukan data yang paling relevan dari kumpulan data besar yang telah diindeks sebelumnya.

- **`index.search(query_embedding, k)`**  
  Melakukan pencarian terhadap *k* tetangga terdekat (lowongan kerja) yang memiliki jarak paling kecil terhadap vektor kueri pengguna.  
  Fungsi ini mengembalikan dua keluaran:
  - **D (Distance):** Nilai jarak antar vektor sebagai ukuran ketidakmiripan.
  - **I (Index):** Indeks baris data yang paling relevan di dalam indeks FAISS.

- **Pemetaan ke Metadata**  
  Indeks hasil pencarian (`idx`) digunakan untuk mengambil informasi lengkap lowongan kerja—seperti judul, deskripsi, dan keterampilan—dari objek `metadata` yang telah disimpan sebelumnya.

- **Skor Kemiripan (*Similarity Score*)**  
  Perhitungan `1 / (1 + dist)` digunakan untuk mengonversi jarak L2 menjadi skor kemiripan dalam rentang 0 hingga 1.  
  Semakin mendekati nilai 1, semakin tinggi tingkat kemiripan antara kueri pengguna dan data yang ditemukan.

- **`results`**  
  Menghasilkan daftar lowongan kerja yang telah diurutkan berdasarkan tingkat relevansi tertinggi, sehingga dapat langsung digunakan sebagai hasil rekomendasi bagi pengguna.

In [27]:
def search(query_embedding, k=10):
        D, I = index.search(
            query_embedding, k
        )

        results = []
        for dist, idx in zip(D[0], I[0]):
            job = metadata[idx].copy()
            job["distance"] = float(dist)
            job["similarity"] = float(1 / (1 + dist))  # optional
            results.append(job)

        return results

## Fungsi Rekomendasi Peran (Top Roles Recommendation)

Fungsi ini berperan sebagai *wrapper* akhir yang mengintegrasikan seluruh komponen utama—Transformer, FAISS, dan metadata—ke dalam satu alur kerja rekomendasi yang utuh dan siap digunakan.

- **`embed_texts([user_text])`**  
  Mengambil teks mentah dari pengguna, seperti ringkasan CV, pengalaman kerja, atau minat profesional, lalu mengubahnya menjadi satu vektor semantik yang merepresentasikan konteks pengguna secara menyeluruh.

- **Pencarian Otomatis**  
  Vektor pengguna tersebut kemudian diproses melalui fungsi `search` untuk dibandingkan dengan ribuan vektor lowongan kerja yang tersimpan di dalam indeks FAISS, sehingga proses pencocokan dapat dilakukan secara cepat dan efisien.

- **Parameter `k=3`**  
  Menentukan jumlah hasil teratas yang dikembalikan. Secara default, fungsi ini akan merekomendasikan **tiga peran atau lowongan kerja** yang paling relevan dengan profil pengguna.

- **Output Komprehensif**  
  Menghasilkan daftar rekomendasi berupa objek pekerjaan lengkap—meliputi judul pekerjaan, deskripsi, keterampilan yang dibutuhkan—beserta skor kemiripan yang merefleksikan tingkat kecocokan dengan input pengguna.

In [28]:
def get_top_roles_by_text(user_text, k=3):
    emb = embed_texts([user_text])
    results = search(query_embedding=emb, k=k)
    return results

## Analisis Pencarian Rekomendasi

Tahap ini merupakan bagian akhir dari sistem, di mana profil pengguna dicocokkan dengan basis data lowongan kerja melalui mekanisme pencarian berbasis vektor. Berikut alur proses yang terjadi secara internal:

1. **Vektorisasi Kueri**  
   Teks `persona_9_sales_marketing` diubah menjadi representasi vektor numerik menggunakan model `paraphrase-multilingual-MiniLM-L12-v2`, sehingga makna semantik teks dapat diproses secara matematis.

2. **Perhitungan Jarak (L2 Distance)**  
   FAISS membandingkan vektor kueri dengan seluruh vektor lowongan kerja yang tersimpan di dalam indeks, kemudian mencari lima vektor dengan jarak Euclidean (L2) paling kecil sebagai kandidat terdekat.

3. **Pengurutan Hasil**  
   Kandidat yang ditemukan diurutkan berdasarkan skor kemiripan tertinggi, yaitu vektor dengan jarak paling dekat terhadap kueri pengguna. Hasil teratas merepresentasikan lowongan kerja yang paling relevan dengan profil pengguna.

In [29]:
rekomendasi = get_top_roles_by_text(persona_9_sales_marketing[0], k=5)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
rekomendasi

[{'id': 20322,
  'job_title': 'Senior Sales Manager',
  'job_function': 'Hotel/Restoran,Hotel/Pariwisata',
  'skills': "['marketing strategy', 'sale presentation', 'accounts', 'creative', 'sales', 'coordinates', 'advertisers', 'production managers', 'plans', 'economic', 'surveys']",
  'job_description': 'Identify, develops, and evaluates marketing strategy, based on knowledge of establishment objectives, market characteristics, and cost and mark up factors.Assists in preparation, administration and documentation of proposals.Maintains existing business accounts and secures new accounts through aggressive and creative sales & marketing programmes.Conducts sales presentations to prospective clients.Coordinates and participates in promotional activities and trade shows, working with developers, advertisers, and production managers, to market the serviced apartment.Prepares monthly reports of leasing for review of plans and procedures.Conducts economic and commercial surveys to identify po

# Skill Gap Module

### Analisis Kepentingan Skill (TF-IDF Global Importance)

Bagian ini bertujuan untuk mengukur tingkat kepentingan atau kekhasan setiap *skill* di seluruh dataset menggunakan pendekatan statistik berbasis TF-IDF.

- **`TfidfVectorizer(ngram_range=(1, 2))`**  
  Mengekstraksi kata tunggal (misalnya *"python"*) dan frasa dua kata (misalnya *"data analysis"*). Metode TF-IDF memberikan bobot tinggi pada skill yang bersifat spesifik dan informatif, serta bobot lebih rendah pada kata-kata umum yang muncul di hampir semua lowongan, seperti *"kerja"* atau *"tim"*.

- **`tfidf_matrix.mean(axis=0)`**  
  Menghitung nilai rata-rata bobot TF-IDF dari seluruh dokumen untuk setiap fitur. Hasil ini merepresentasikan **Global Importance**, yaitu tingkat signifikansi suatu skill secara keseluruhan dalam kumpulan data lowongan kerja.

- **`global_importance_dict`**  
  Menyusun hasil perhitungan ke dalam bentuk kamus (*dictionary*) yang memetakan setiap skill ke skor kepentingannya, sehingga mudah dianalisis atau digunakan pada tahap selanjutnya, seperti visualisasi atau rekomendasi berbasis skill.

In [31]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
skills_data = df['skills'].astype(str).fillna('')
tfidf.fit(skills_data)

# Siapkan Global Importance 
tfidf_matrix = tfidf.transform(skills_data)
global_scores = np.asarray(tfidf_matrix.mean(axis=0)).flatten()
feature_names = tfidf.get_feature_names_out()

# Buat dictionary: { 'nama_skill': skor_kepentingan }
global_importance_dict = dict(zip(feature_names, global_scores))

Dalam modul rekomendasi skill (Skill Gap Module), sistem tidak menggunakan rata-rata aritmatika biasa untuk memberikan skor rekomendasi. Sebaliknya, kami menerapkan Rerata Harmonis (Harmonic Mean), sebuah pendekatan yang sama dengan perhitungan F1-Score dalam evaluasi klasifikasi.

Skor akhir ($S$) untuk setiap kandidat skill dihitung berdasarkan dua komponen:

1. Relevance ($R$): Seberapa besar "kekosongan" (gap) skill ini pada target pekerjaan yang dituju user? (Dihitung dari selisih vektor TF-IDF).
2. Importance ($I$): Seberapa vital skill ini secara global menurut model Random Forest? (Dihitung dari Feature Importance).

Harmonic Mean Logic

$$Score = \frac{2 \times (Relevance \times Importance)}{Relevance + Importance}$$

Dengan menggunakan Harmonic Mean, sistem menjamin bahwa skill yang direkomendasikan adalah skill yang benar-benar "High Impact" secara global, namun tetap "Contextually Accurate" dengan tujuan karir user.

In [32]:
class SkillGapModule:
    def __init__(self, vectorizer, df, global_imp_dict):
        self.vectorizer = vectorizer
        self.feature_names = vectorizer.get_feature_names_out()
        self.global_imp = global_imp_dict
        self.df = df 
        
    def _clean_skill_string(self, skill_str):
        """
        Fungsi untuk membersihkan string skill
        """
        try:
            # Jika bentuknya string list python
            if skill_str.startswith("[") and skill_str.endswith("]"):
                skill_list = ast.literal_eval(skill_str)
                return " ".join(skill_list)
            return skill_str
        except:
            return skill_str

    def get_target_vector_from_recs(self, recommendation_list):
        """
        Membuat vektor target berdasarkan RATA-RATA skill dari top rekomendasi.
        """
        if not recommendation_list:
            return None
        
        # Ambil skill dari setiap job di rekomendasi
        cleaned_skills = []
        for job in recommendation_list:
            raw_skill = str(job.get('skills', ''))
            cleaned_text = self._clean_skill_string(raw_skill)
            cleaned_skills.append(cleaned_text)
            
        # Buat Matrix TF-IDF untuk job-job rekomendasi ini
        target_vec_matrix = self.vectorizer.transform(cleaned_skills)
        
        # Hitung Centroid (Rata-rata vektor)
        # Ini merepresentasikan "Skill Ideal" untuk cluster rekomendasi ini
        composite_vector = np.asarray(target_vec_matrix.mean(axis=0)).flatten()
        
        return composite_vector

    def analyze(self, user_text, recommendation_list, top_n=10):
        """
        user_text: Text input dari user (persona)
        recommendation_list: List dictionary hasil search() atau get_top_roles_by_text()
        """
        # Dapatkan Vektor Target dari Rekomendasi
        target_vec = self.get_target_vector_from_recs(recommendation_list)
        
        if target_vec is None:
            return []
        
        # Hitung Vektor User (Skill yang user punya)
        user_vec = self.vectorizer.transform([user_text]).toarray().flatten()
        
        # Hitung Gap (Target - User) -> Ambil yang positif saja (Missing Skills)
        gap_vec = np.maximum(0, target_vec - user_vec)
        
        # Threshold filter noise (agar skill dengan bobot sangat kecil tidak muncul)
        gap_indices = np.where(gap_vec > 0.005)[0]
        
        if len(gap_indices) == 0:
            return [] # Tidak ada gap signifikan

        # Scoring Kandidat Skill
        candidates = []
        for idx in gap_indices:
            skill_name = self.feature_names[idx]
            
            # RELEVANCE: Seberapa besar gap skill ini di job rekomendasi?
            relevance = gap_vec[idx]
            
            # IMPORTANCE: Seberapa penting skill ini secara global (Model MLP)?
            importance = self.global_imp.get(skill_name, 0)
            
            candidates.append({
                'skill': skill_name,
                'relevance': relevance,
                'importance': importance
            })
            
        if not candidates:
            return []
            
        # Normalisasi & Harmonic Mean (F1-like Score)
        df_cand = pd.DataFrame(candidates)
        scaler = MinMaxScaler()
        
        if len(df_cand) > 1:
            scaled_vals = scaler.fit_transform(df_cand[['relevance', 'importance']])
            df_cand['rel_norm'] = scaled_vals[:, 0]
            df_cand['imp_norm'] = scaled_vals[:, 1]
        else:
            df_cand['rel_norm'] = 1.0
            df_cand['imp_norm'] = 1.0
            
        epsilon = 1e-5
        # Rumus Harmonic Mean: Mengutamakan skill yang Relevan DAN Penting
        df_cand['final_score'] = 2 * (df_cand['rel_norm'] * df_cand['imp_norm']) / \
                                     (df_cand['rel_norm'] + df_cand['imp_norm'] + epsilon)
        
        # Ambil Top N Score Tertinggi
        results = df_cand.sort_values('final_score', ascending=False).head(top_n)
        
        return results[['skill', 'final_score']].to_dict('records')

# Inisiasi Modul
gap_analyzer = SkillGapModule(tfidf, df, global_importance_dict)

### Alur Kerja Terintegrasi

Bagian ini menjelaskan alur end-to-end sistem rekomendasi dan analisis kesenjangan kompetensi yang dibangun, mulai dari pencarian semantik hingga penentuan prioritas pembelajaran.

1. **Semantic Retrieval**  
   Fungsi `get_top_roles_by_text` memanfaatkan model Transformer untuk mencari lowongan kerja yang paling relevan secara semantik terhadap narasi `user_persona`. Pendekatan ini melampaui pencarian berbasis kata kunci karena mempertimbangkan konteks dan makna keseluruhan teks.

2. **Sintesis Profil Ideal**  
   Modul `gap_analyzer` mengolah lima lowongan teratas hasil pencarian dan menyusunnya menjadi satu representasi profil kompetensi ideal. Proses ini dilakukan menggunakan teknik *centroid vectoring* untuk menangkap karakteristik umum yang paling dominan dari peran-peran tersebut.

3. **Analisis Kesenjangan Kompetensi**  
   Sistem menghitung selisih vektor antara profil ideal dan representasi vektor pengguna. Hasil pengurangan ini menghasilkan daftar keahlian yang belum tercakup dalam profil pengguna, tetapi memiliki tingkat permintaan tinggi di pasar kerja.

4. **Prioritas Pembelajaran**  
   Skor akhir (`final_score`) digunakan untuk memeringkat keahlian berdasarkan urgensi dan dampaknya. Keahlian dengan skor tertinggi merepresentasikan *critical needs* atau *low-hanging fruit*—kompetensi yang relatif strategis untuk dipelajari lebih dahulu karena berpotensi memberikan peningkatan peluang kerja yang signifikan.

In [33]:
user_persona = persona_9_sales_marketing[0]
print(f"User Input: {user_persona[:100]}...")
rekomendasi = get_top_roles_by_text(user_persona, k=5)

# Tampilkan Judul Job yang direkomendasikan
print("\nRekomendasi Pekerjaan:")
for job in rekomendasi:
    print(f"- {job['job_title']} (Similarity: {job['similarity']:.3f})")

# Analisis Skill Gap
skill_gaps = gap_analyzer.analyze(user_persona, rekomendasi, top_n=10)

print("\n--- SKILL GAP ANALYSIS ---")
if not skill_gaps:
    print("Selamat! Skill kamu sangat cocok, tidak ada gap signifikan.")
else:
    print("Skill yang perlu dipelajari untuk pekerjaan di atas:")
    for i, gap in enumerate(skill_gaps, 1):
        print(f"{i}. {gap['skill']} (Score: {gap['final_score']:.4f})")

User Input: 
Saya memiliki pengalaman di bidang sales dan marketing, dengan fokus pada pengembangan pasar dan pe...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Rekomendasi Pekerjaan:
- Senior Sales Manager (Similarity: 0.185)
- SALES MANAGER - RETAIL (Similarity: 0.175)
- Sales Head Automotive Roda 2 (Similarity: 0.171)
- Admin online shop (Ecommerce) (Similarity: 0.165)
- Sales Marketing (F&B) (Similarity: 0.159)

--- SKILL GAP ANALYSIS ---
Skill yang perlu dipelajari untuk pekerjaan di atas:
1. bahasa (Score: 0.4697)
2. customer (Score: 0.4498)
3. data (Score: 0.3950)
4. manager (Score: 0.3410)
5. social (Score: 0.3380)
6. plan (Score: 0.3353)
7. leadership (Score: 0.3321)
8. medium (Score: 0.3065)
9. social medium (Score: 0.2899)
10. communication (Score: 0.2551)


In [34]:
joblib.dump(tfidf, "tfidf.pkl")
joblib.dump(global_importance_dict, "global_importance.pkl")

['global_importance.pkl']

## Buat Opsi Skill Gap Direct

Skill Gap Module sebelumnya menghimpun skill yang dibutuhkan secara global, untuk opsi kedua kali ini skill gap module dibuat secara sederhana sesuai dari himpunan skill yang berasal dari rekomendasi job.

In [35]:
class DirectSkillGapModule:
    def __init__(self, vectorizer):
        # Kita hanya butuh vectorizer (TF-IDF) untuk menerjemahkan teks ke angka
        self.vectorizer = vectorizer
        self.feature_names = vectorizer.get_feature_names_out()
        
    def _clean_skill_string(self, skill_str):
        """Membersihkan format string list"""
        try:
            if isinstance(skill_str, str) and skill_str.strip().startswith("["):
                skill_list = ast.literal_eval(skill_str)
                return " ".join(skill_list)
            return str(skill_str)
        except:
            return str(skill_str)

    def get_target_vector_from_recs(self, recommendation_list):
        """
        Membuat 'Profil Skill Ideal' berdasarkan rata-rata skill 
        dari pekerjaan yang direkomendasikan saja.
        """
        if not recommendation_list:
            return None
        
        cleaned_skills = []
        for job in recommendation_list:
            raw_skill = job.get('skills', '')
            cleaned_text = self._clean_skill_string(raw_skill)
            cleaned_skills.append(cleaned_text)
            
        if not cleaned_skills:
            return None

        # Buat centroid (rata-rata vector)
        target_vec_matrix = self.vectorizer.transform(cleaned_skills)
        composite_vector = np.asarray(target_vec_matrix.mean(axis=0)).flatten()
        
        return composite_vector

    def analyze(self, user_text, recommendation_list, top_n=10):
        # Tentukan Target (Rata-rata skill dari Rekomendasi)
        target_vec = self.get_target_vector_from_recs(recommendation_list)
        if target_vec is None: 
            return []
        
        # Tentukan Posisi User (Skill yang sudah dimiliki)
        user_vec = self.vectorizer.transform([user_text]).toarray().flatten()
        
        # Hitung Gap Murni (Target - User)
        gap_vec = np.maximum(0, target_vec - user_vec)
        
        # Filter: Hanya ambil gap yang nilainya signifikan (> 0.01)
        gap_indices = np.where(gap_vec > 0.01)[0]
        
        if len(gap_indices) == 0:
            return []

        # Susun Hasil
        candidates = []
        for idx in gap_indices:
            skill_name = self.feature_names[idx]
            gap_score = gap_vec[idx] # Semakin besar skor, semakin urgent skill ini
            
            candidates.append({
                'skill': skill_name,
                'gap_score': gap_score
            })
            
        # Urutkan berdasarkan Gap Terbesar
        df_gap = pd.DataFrame(candidates)
        results = df_gap.sort_values('gap_score', ascending=False).head(top_n)
        
        return results.to_dict('records')

# Inisiasi Modul
direct_gap_analyzer = DirectSkillGapModule(tfidf)

## Alur Kerja Pipeline

1. **Pencocokan Semantik**  
   Fungsi `get_top_roles_by_text` memanfaatkan representasi vektor berdimensi tinggi untuk menemukan lowongan kerja yang paling relevan secara makna dengan profil pengguna, meskipun kata kunci yang digunakan tidak identik.

2. **Agregasi Skill**  
   Sistem mengumpulkan seluruh skill dari lima lowongan kerja teratas untuk membentuk gambaran kebutuhan kompetensi industri yang sedang dominan.

3. **Pengurangan Vektor (Vector Subtraction)**  
   Modul `analyze` mengidentifikasi fitur atau skill yang memiliki bobot tinggi pada lowongan kerja, tetapi memiliki bobot rendah atau tidak muncul sama sekali pada profil pengguna.

4. **Penilaian Kesenjangan (Gap Scoring)**  
   Nilai `gap_score` memberikan ukuran kuantitatif tingkat urgensi setiap skill. Semakin tinggi skornya, semakin penting skill tersebut untuk segera dipelajari agar profil pengguna menjadi lebih kompetitif dibandingkan kandidat lain pada lowongan yang dianalisis.

In [36]:
user_persona = persona_9_sales_marketing[0]
print(f"User Input: {user_persona[:100]}...")

# Cari Rekomendasi Pekerjaan dulu (Pakai fungsi search kamu)
rekomendasi = get_top_roles_by_text(user_persona, k=5)

# Cari Gap Skill
gaps = direct_gap_analyzer.analyze(user_persona, rekomendasi)

print(f"User Persona: {user_persona}")
print("\n--- REKOMENDASI PEKERJAAN ---")
for job in rekomendasi:
    print(f"- {job['job_title']}")

print("\n--- SKILL GAP (PRIORITAS UNTUK DIPELAJARI) ---")
if not gaps:
    print("Tidak ada gap skill signifikan.")
else:
    for i, item in enumerate(gaps, 1):
        # Score menggambarkan seberapa sering skill ini muncul di rekomendasi
        # tapi hilang di user.
        print(f"{i}. {item['skill']} (Gap Score: {item['gap_score']:.4f})")

User Input: 
Saya memiliki pengalaman di bidang sales dan marketing, dengan fokus pada pengembangan pasar dan pe...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

User Persona: 
Saya memiliki pengalaman di bidang sales dan marketing, dengan fokus pada pengembangan pasar dan pencapaian target penjualan. Saya terbiasa 
membangun hubungan dengan pelanggan, memahami kebutuhan mereka, serta menawarkan solusi yang sesuai. Saya memiliki kemampuan komunikasi dan 
negosiasi yang baik. Saya sering terlibat dalam perencanaan strategi pemasaran, pelaksanaan kampanye, serta evaluasi hasil penjualan. Saya memahami
pentingnya analisis pasar dan perilaku pelanggan dalam menentukan strategi yang efektif. Saya tertarik untuk terus berkembang di bidang sales dan 
marketing dengan fokus pada pertumbuhan bisnis dan kepuasan pelanggan.


--- REKOMENDASI PEKERJAAN ---
- Senior Sales Manager
- SALES MANAGER - RETAIL
- Sales Head Automotive Roda 2
- Admin online shop (Ecommerce)
- Sales Marketing (F&B)

--- SKILL GAP (PRIORITAS UNTUK DIPELAJARI) ---
1. customer (Gap Score: 0.1037)
2. sales marketplace (Gap Score: 0.0924)
3. broadcast (Gap Score: 0.0801)
4. medium ide (G